<a href="https://colab.research.google.com/github/Vikram-Ahuja/Stock_Portfolios/blob/main/Nifty_Monentum_Portfolio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import requests as rq
from bs4 import BeautifulSoup
import datetime as dt
from scipy import stats
from statistics import mean

In [ ]:
Nifty_data = pd.read_csv("/content/drive/MyDrive/Nifty_Stocks_data.csv")

Nifty_Data_sorted = Nifty_data.sort_values("One_Year_Return", ascending=False,ignore_index=True)

Nifty_Portfolio_15 = Nifty_Data_sorted[:15]

Nifty_Portfolio_15.to_csv("/content/drive/MyDrive/Nifty_Portfolio.csv")

In [ ]:
stocks = pd.read_excel("/content/drive/MyDrive/NIfty_stocks.xlsx")
stocks_month_price ={}
for stock in stocks.Symbols: 
    d = pd.read_html(f"https://in.finance.yahoo.com/quote/{stock}.NS/history?period1=1514764800&period2=1615334400&interval=1mo&filter=history&frequency=1mo&includeAdjustedClose=true")
    d = d[0][:30]
    
    del d['Volume']
    del d['Close*']
    d = d.rename(columns={"Adj. close**": "Close"})
    stocks_month_price[stock]  = d 

    for i in range(len(stocks_month_price[stock])):
        x = stocks_month_price[stock].Date[i]
        if "01-" not in x:
            stocks_month_price[stock].drop([i], inplace = True)

    stocks_month_price[stock].reset_index(drop=True,inplace =True) 
     


In [ ]:
def return_calculator(stock,time):
    try:
        x = float(stocks_month_price[stock].Close[time])
        y = float(stocks_month_price[stock].Close[0])
    except:
        x = float(stocks_month_price[stock].Close[time+1])
        y = float(stocks_month_price[stock].Close[1])
    return_ = (y-x)*100/x
    percent = round(return_ , 4)
    return percent

def google_price_scrape(stock):
    url = f"https://www.google.com/finance/quote/{stock}:NSE"
    data = rq.get(url).text
    soup = BeautifulSoup(data,"lxml")
    price1 = soup.find("div",class_ = "YMlKec fxKbKc")
    price2 = price1.text.replace(",","")
    price = price2[1:]
    return price

In [ ]:
hqm_column = ["Symbol" , "Price" , "One_Year_return","One_Year_return_percentile" , "Six_month_return","Six_month_return_percentile","Three_month_return","Three_month_return_percentile","One_month_return","One_month_return_percentile","HQM_Score"]
hqm_Nifty_50_Porfolio_15 = pd.DataFrame(columns=hqm_column)

for stock in stocks.Symbols:
    price = google_price_scrape(stock)
    One_Year_return = return_calculator(stock,12)
    Six_month_return = return_calculator(stock,6)
    Three_month_return = return_calculator(stock,3)
    One_month_return =return_calculator(stock,1)
    hqm_Nifty_50_Porfolio_15 = hqm_Nifty_50_Porfolio_15.append(
        pd.Series(
            [
            stock,
            price,
            One_Year_return,
            "N/A", 
            Six_month_return,
            "N/A",
            Three_month_return,
            "N/A",
            One_month_return,
            "N/A",
            "N/A"
            ],
            index =hqm_column
        ),
        ignore_index = True
    )


In [ ]:
hqm_

In [ ]:
time_period = ["One_Year","Six_month","Three_month","One_month"]

for row in hqm_Nifty_50_Porfolio_15.index:
    for time in time_period:
        hqm_Nifty_50_Porfolio_15.at[row,f"{time}_return_percentile"] = float(stats.percentileofscore(hqm_Nifty_50_Porfolio_15[f"{time}_return"],hqm_Nifty_50_Porfolio_15.at[row,f"{time}_return"]))

In [ ]:
for row in hqm_Nifty_50_Porfolio_15.index:
    l = []
    for time in time_period:
        l.append(hqm_Nifty_50_Porfolio_15.at[row,f"{time}_return_percentile"])
    avg = mean(l)
    hqm_Nifty_50_Porfolio_15.at[row,"HQM_Score"] = avg

In [ ]:
Nifty_Portfolio_HQM_15 = hqm_Nifty_50_Porfolio_15.sort_values("HQM_Score",ascending=False,ignore_index=True)[:15]

Nifty_Portfolio_HQM_15.to_csv("/content/drive/MyDrive/Nifty_Portfolio_Momentum.csv")

In [ ]:
Nifty_Portfolio_HQM_15